In [4]:
import cbstools as cbs
from os.path import join
import os
import nibabel as nb
import numpy as np
import glob as glob
import pandas as pd

In [5]:
current_dir = os.getcwd() #current directy of this notebook

data_dir='/home/chris/Documents/code/python/cbstools-python/test-python'
out_dir='/home/chris/Documents/code/python/cbstools-python/test-python/out'

t1_fname='t1map_stripped.nii.gz'
uni_fname='uni_stripped.nii.gz'
pre_fname='filters.nii.gz'

atlas_file = 'brain-atlas-3.0.3.txt'

print(cbs.ATLAS_DIR) #as defined in your defaults file
print(cbs.TOPOLOGY_LUT_DIR)

/home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/
/home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/


In [3]:
#cbs=reload(cbs)
cbs.setup_JVM()

Java virtual machine successfully started.


In [14]:
cbs.MGDMBrainSegmentation([[join(data_dir,uni_fname),"MP2RAGE7T"]],
                             output_dir=out_dir,atlas_file=None, topology_lut_dir=None)

Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...
Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
  1  ['/home/chris/Documents/code/python/cbstools-python/test-python/uni_stripped.nii.gz', 'MP2RAGE7T']
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/code/python/cbstools-python/test-python/out/uni_stripped_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/code/python/cbstools-python/test-python/out
Execution completed


In [15]:
cbs=reload(cbs)
cbs.MGDMBrainSegmentation([[join(data_dir,uni_fname),"MP2RAGE7T"],[join(data_dir,pre_fname),"Filters"]],
                             output_dir=out_dir,atlas_file=None, topology_lut_dir=None)


Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
  1  ['/home/chris/Documents/code/python/cbstools-python/test-python/uni_stripped.nii.gz', 'MP2RAGE7T']
Input files and filetypes:
  2  ['/home/chris/Documents/code/python/cbstools-python/test-python/filters.nii.gz', 'Filters']
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/code/python/cbstools-python/test-python/out/uni_stripped_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/code/python/cbstools-python/test-python/out
Execution completed


In [69]:
## test priors updating
cbs=reload(cbs)
seg_file = glob.glob(os.path.join(out_dir,"*_seg_cjs.nii.gz"))
seg_files = [seg_file[0],seg_file[0]]
met_files = [join(data_dir,uni_fname),join(data_dir,uni_fname)]
met_con="Mprage3T"
[medians,spreads]=cbs.generate_group_intensity_priors(seg_files,met_files,"bob",
                                                                      join(cbs.ATLAS_DIR,atlas_file))

You have not chosen a valid contrast for your metric_contrast_name, please choose from: 
DWIFA3T, DWIMD3T, T1map9T, Mp2rage9T, T1map7T, Mp2rage7T, PV, Filters, T1pv, Mprage3T, T1map3T, Mp2rage3T, HCPT1w, HCPT2w, NormMPRAGE


In [67]:
medians

--- after here is mockup for creating a scriptable change in priors --- (it works!)

In [487]:
def seg_erode(seg_d, iterations=1, background_idx = 1,
              structure=None, min_vox_count = 5,seg_null_value = 0,
             VERBOSE=False):
    # erode indices (integers) to identify "core" structure
    # XXX might need to limit erosion here and loop myself
    # default erosion structure is 3,1 (which is not super restrictive, and should work for most)
    # seg null value is int value that is assigned to voxels that were eroded from the segmentation
    import scipy.ndimage as ndi
    import numpy as np
    
    if structure is None:
        structure = ndi.morphology.generate_binary_structure(3, 1)
    if seg_null_value == 0:
        seg_shrunk_d = np.zeros_like(seg_d)
        temp_d = np.zeros_like(seg_d)
    else:
        seg_shrunk_d = np.ones_like(seg_d)*seg_null_value
        temp_d = np.ones_like(seg_d)*seg_null_value    
    
    seg_idxs = np.unique(seg_d)
  
    for seg_idx in seg_idxs:
        print(seg_idx),
        if (background_idx is not None) and (background_idx == seg_idx):
            seg_shrunk_d[seg_d==seg_idx] = seg_idx #just set the value to the bckgrnd value, and be done with it
            if VERBOSE:
                print("[bckg]"),
        else:
            temp_d[seg_d==seg_idx] = 1
            for idx in range(0, iterations): #messy, does not exit the loop when already gone too far.
                temp_temp_d = ndi.binary_erosion(temp_d, iterations=1, structure=structure)
                if np.sum(temp_temp_d) >= min_vox_count:
                    temp_d = temp_temp_d
                    if VERBOSE:
                        print("[y]"),
                else:
                    if VERBOSE:
                        print("[no]"),
            seg_shrunk_d[temp_d==1] = seg_idx
            temp_d[:,:,:] = seg_null_value
            if VERBOSE:
                print(seg_idx)
    print("")
    return seg_shrunk_d



def extract_metrics_from_seg(seg_d, metric_d, norm_data = True,
                             background_idx = 1, seg_null_value = 0,
                             percentile_top_bot = [75,25],
                             return_normed_metric_d=False):
    #returns np matrix of indices, and one of median, and percentiles
    #norm_data = true first zscores all of the data other than the background
    import numpy as np
    import scipy
    seg_idxs = np.unique(seg_d)
    res = np.zeros((len(seg_idxs),3))
    
    if norm_data: # rescale the data to 0
        if background_idx is not None: #we need to exclude the background data from the norming
            metric_d[seg_d!=background_idx] = (metric_d[seg_d!=background_idx] - np.min(metric_d[seg_d!=background_idx])) / (np.max(metric_d[seg_d!=background_idx]) - np.min(metric_d[seg_d!=background_idx]))
        else:
            metric_d = (metric_d - np.min(metric_d)) / (np.max(metric_d)-np.min(metric_d))
        
    for idx,seg_idx in enumerate(seg_idxs):
        if (background_idx is not None) and ((seg_idx == background_idx) or (seg_idx == seg_null_value)):
            res[idx,:] = [0,0,0]
        else:
            d_1d = np.ndarray.flatten(metric_d[seg_d == seg_idx])
            res[idx,:] = [np.mean(d_1d),
                          np.percentile(d_1d,np.max(percentile_top_bot)),
                          np.percentile(d_1d,np.min(percentile_top_bot))]
    if return_normed_metric_d:
        return seg_idxs,res,metric_d
    else:
        return seg_idxs,res

In [485]:
# load the output segmentatinb.Nifti1Image(d_seg_ero,a_seg).to_filename(os.path.join(out_dir,"d_seg_ero.nii.gz"))on file
seg_files = glob.glob(os.path.join(out_dir,"*_seg_cjs.nii.gz"))
# point it back at the input file, extract values from 
seg_file=seg_files[0]
img=nb.load(seg_file)
d_seg = img.get_data()
a_seg = img.affine

d_metric = nb.load(join(data_dir,uni_fname)).get_data()
seg_null_value = 0

print("seg indices: "),
np.unique(d_seg)
np.where(d_seg==50)
#d_seg[70,66,43]

seg indices: 

(array([ 14,  14,  14, ..., 132, 132, 132]),
 array([ 86,  87,  87, ..., 103, 103, 103]),
 array([60, 51, 58, ..., 57, 58, 59]))

In [489]:
#erode the seg file
d_seg_ero = seg_erode(d_seg,background_idx = 1, seg_null_value = seg_null_value)
nb.Nifti1Image(d_seg_ero,a_seg).to_filename(os.path.join(out_dir,"d_seg_ero.nii.gz"))

#extract the values from each index
[seg_idxs,seg_stats,d_metric_norm] = extract_metrics_from_seg(d_seg_ero,d_metric,seg_null_value=seg_null_value,
                                                             return_normed_metric_d=True)
nb.Nifti1Image(d_metric_norm,a_seg).to_filename(os.path.join(out_dir,"d_metric_norm.nii.gz"))

1 8 10 11 12 13 14 17 18 20 25 26 27 30 31 32 33 34 35 36 37 38 39 40 41 43 46 47 48 50 


In [513]:
## explicitly set our median and spread variables for priors, just makes it easier to put into df later

MIN_QUART_DIFF = 0.10
prior_medians=seg_stats[:,0]
prior_quart_diffs=np.squeeze(np.abs(np.diff(seg_stats[:,1:3])))

# fill in prior_quart_diffs that are below a certain value??
prior_quart_diffs[prior_quart_diffs<MIN_QUART_DIFF] = MIN_QUART_DIFF
print(seg_idxs)
print(prior_medians)
print(prior_quart_diffs)



[ 0  1  8 10 11 12 13 14 17 18 20 25 26 27 30 31 32 33 34 35 36 37 38 39 40
 41 43 46 47 48 50]
[ 0.          0.          0.          0.22563398  0.08410583  0.08987724
  0.16868669  0.20622344  0.48463598  0.17966332  0.2674751   0.46390796
  0.45047241  0.46238199  0.39889997  0.34720081  0.48333097  0.46535555
  0.35564139  0.36721775  0.56746542  0.54524189  0.68753362  0.65726626
  0.78753018  0.7920326   0.69178802  0.75529468  0.8249054   0.83420277
  0.06309689]
[ 0.1         0.1         0.1         0.24719246  0.1         0.1
  0.18838063  0.18998904  0.1         0.1         0.30075012  0.21800042
  0.32179964  0.29512179  0.17577323  0.18397549  0.1         0.10100538
  0.15619811  0.15573829  0.13367768  0.13912316  0.12773302  0.1336633
  0.1         0.1         0.11633818  0.1         0.1         0.1         0.1       ]


In [561]:
## parse the atlas file to get the lut and the intensity priors of interest
new_atlas_file = 'brain-atlas-3.0.3_cjs.txt'
lut_idx=np.zeros(2).astype(int)
con_idx=np.zeros(2).astype(int)
contrast_name = 'Mprage3T'

#identify the start and stop locations for the LUT and the intensity priors of interest
fp = open(os.path.join(cbs.ATLAS_DIR,atlas_file))
for i, line in enumerate(fp):
    if "Structures:" in line: #this is the beginning of the LUT
        lut_idx[0] = i
    if "Topology Atlas:" in line: #the end of the LUT
        lut_idx[1] = i-2
    if "Intensity Prior:" in line:
        if contrast_name in line:
            con_idx[0] = i
fp.close()

lut=pd.read_csv(os.path.join(cbs.ATLAS_DIR,atlas_file),sep="\t+",
                skiprows=lut_idx[0]+1,nrows=lut_idx[1]-lut_idx[0],engine='python',
                names=["Index","Type"])

con_idx[1] = len(lut) #total number is the same length as the lut
priors=pd.read_csv(os.path.join(cbs.ATLAS_DIR,atlas_file),sep="\t+",
                   skiprows=con_idx[0]+1,nrows=con_idx[1],engine='python',
                   names=["Median","Spread","Weight"])


In [558]:
## create a new dataframe and populate with our new values
# could also add intelligence here, to only push it to 1/2 of the difference etc (next release...)
priors_new = pd.DataFrame.copy(priors)
for idx in lut.Index:
    priors_new[lut["Index"]==idx] = [prior_medians[seg_idxs==idx], prior_quart_diffs[seg_idxs==idx],1]
priors_new.head()
priors_new_string = priors_new.to_csv(sep="\t",header=False,float_format="%.2f")
priors_new_string_lines = priors_new_string.split("\n")[0:-1] #convert to list of lines, cut the last empty '' line

In [578]:
#identify the start and stop locations for the LUT and the intensity priors of interest
fp = open(os.path.join(cbs.ATLAS_DIR,atlas_file))
fp_new = open(os.path.join(cbs.ATLAS_DIR,new_atlas_file),"w")
ii=0
#only replace the lines that we changed
for i, line in enumerate(fp):
    if i > con_idx[0] and i<con_idx[1]+con_idx[0]:
        fp_new.write(priors_new_string_lines[ii]+"\n")
        ii+=1
    else:
        fp_new.write(line)
fp.close()
fp_new.close()
print("New atlas file written to: " + fp_new.name)

New atlas file written to: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3_cjs.txt
